# Importing Required Libraries


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd

# Importing Dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("Training data shape : ", X_train.shape)
print("Training labels shape : ", y_train.shape)
print("Testing data shape : ", X_test.shape)
print("Testing labels shape : ", y_test.shape)

In [ ]:
y_test

# Constants

In [ ]:
n_classes = 10
epochs = 10
fine_label_list = ['0','1','2','3','4','5','6','7','8','9']
input_shape = (28,28)

# Preprocessing

In [ ]:
def unison_shuffled_copies(a, b):
  randomize = np.arange(len(a))
  np.random.shuffle(randomize)
  x = a[randomize]
  y = b[randomize]
  return x, y

# Shuffling dataset
X_train, y_train = unison_shuffled_copies(X_train, y_train)
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

# Visualizing Dataset

In [ ]:
num_classes= 10 # Number of classes visualized
num_samples= 3 # Number of samples from each class
class_plotted = [int(i) for i in fine_label_list]

for i in range(0,10,2):
    image_samples = X_train[y_train.reshape(-1) == class_plotted[i]][:num_samples]
    fig, ax1 = plt.subplots(nrows = 1, ncols = num_samples*2, figsize=(8,2) )
    #fig.suptitle("label : %d   label : %d" % (class_plotted[i],
    #                                          class_plotted[i+1]))
    for j in range(num_samples):
        ax1[j].imshow(image_samples[j])
        ax1[j].set_title("label : %d" %(class_plotted[i]))
        ax1[j].axis('off')

    image_samples = X_train[y_train.reshape(-1) == class_plotted[i+1]][:num_samples]
    for j in range(num_samples):
      ax1[num_samples+j].imshow(image_samples[j])
      ax1[num_samples+j].set_title("label : %d" %(class_plotted[i+1]))
      ax1[num_samples+j].axis('off')  

plt.show()

# Creating Network

## Single Layer Perceptron Model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=input_shape),
  tf.keras.layers.Dense(n_classes, activation='softmax')
])

## Multi Layer Perceptron Model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters = 5, kernel_size = (5,5),padding = 'Same',
                 activation ='relu', input_shape = (28,28,1)),
  tf.keras.layers.MaxPool2D( pool_size=(2,2)) ,
  tf.keras.layers.Flatten(input_shape=input_shape),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(n_classes, activation='softmax')]
  )

# Compiling Network

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer='adam', metrics=["accuracy"])
model.summary()

# Training Model

In [ ]:
history = model.fit(x=X_train ,y=y_train, epochs = epochs)

# Training Results

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12,5))
fig.suptitle("Training Performance")
ax1.plot(history.history['accuracy'])
ax2.plot(history.history['loss'])
ax1.set_title("Accuracy")
ax1.set_xlabel("epochs")
ax2.set_title("Loss")
ax2.set_xlabel("epochs")

plt.show()

# Testing Model

In [ ]:
y_test= y_test.reshape(-1)
y_pred = model.predict(X_test)

y_pred2=[]
for i in range(len(y_pred)):
  y_pred2.append( np.argmax(y_pred[i]) )

test_accuracy = accuracy_score(y_true=y_test, y_pred=y_pred2)
test_accuracy *=  100
print(classification_report(y_test, y_pred2))
print("Accuracy: "+str(test_accuracy)+ " %")
print('\n')

# Testing Metrics

In [ ]:
conf_matrix = confusion_matrix(y_true= y_test, y_pred= y_pred2)
                               
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)  
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)
ACC = (TP+TN)/(TP+FP+FN+TN)

df = pd.DataFrame( { 'TP': TP.round(3)
                   , 'TN': TN.round(3)
                   , 'FN': FN.round(3)
                   , 'FP': FP.round(3)
                   , 'Acc': ACC.round(3)}
                  , index = fine_label_list  
                  )
df

# Confusion Matrix

In [ ]:
labels = [int(i) for i in fine_label_list]
cm = confusion_matrix(y_true = y_test, y_pred= y_pred2, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labels)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)
plt.show()

# Visualizing Tests

In [ ]:
n_tests = 5
start = 5
image = X_test[start: start + n_tests]
labels = y_test[start: start + n_tests]
pred = y_pred2[start: start + n_tests]

for i in range(len(labels)):
  fig, (ax1, ax2) = plt.subplots(1, 2 , figsize=(5, 2))

  ax1.axis('off')
  ax1.imshow(image[i] , cmap=plt.cm.gray)
  ax1.set_title("Actual : " + str(fine_label_list[int(labels[i]) ]) )
  ax2.axis('off')
  ax2.imshow(image[i], cmap=plt.cm.gray)
  ax2.set_title('Prediction : '+ str( fine_label_list[int(pred[i] )] ))

plt.show()